In [1]:
import pandas as pd 
import datetime as dt 
from datetime import datetime, timedelta

import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
from sklearn.model_selection import train_test_split
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
cf.set_config_file(offline = True)

In [3]:
def null_counter(df):
    # Check for null values in the whole dataset
    print("Null values in the whole dataset:")
    print(df.isnull().sum())

    # Check for null values for each individual sensor
    sensors = df['Name'].unique()
    for sensor in sensors:
        sensor_df = df[df['Name'] == sensor]
        print(f"\nNull values for sensor {sensor}:")
        print(sensor_df.isnull().sum())

# def convert_todatetime(df):
#     """This function combines the date and time column of a datframe and creates a pandas datetime object"""
#     return pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'], format='%d.%m.%Y %H:%M')

def convert_todatetime(df):
    """This function combines the date and time column of a datframe and creates a pandas datetime object"""

    df['Date'] = df['Date'].astype(str)
    df['Time'] = df['Time'].astype(str)
    df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Date'] = df['Date'] - timedelta(hours=3)
    
    return df
    
def strip_time(df, col='Date'):
    """Covert object to datetime and strip seconds microseconds from the date object"""
    # dt = pd.to_datetime(df[col], format='mixed')
    dt = pd.to_datetime(df[col])
    df[col] = pd.to_datetime(dt.apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))
    
    return df

def date_range(df, col):
    """Taking a dataframe return the start and end date based on a datetime column"""
    return df[col].min(), df[col].max()

def get_frequency(df, thres_mins=10):
    """This function takes a dataframe and datetime column 
    determines the frequency of log values"""

    # Convert the 'Date' column to datetime data type
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')

    # Calculate the time difference between consecutive rows for each sensor
    df['TimeDiff'] = df.groupby('Name')['Date'].diff()

    # Calculate the frequency of logging for each sensor
    sensor_frequency = df.groupby('Name')['TimeDiff'].mean()

    # Format the frequency times to the nearest minutes
    sensor_frequency = pd.to_timedelta(sensor_frequency).round('T')

    # Print the frequency of logging for each sensor
    print("Frequency of logging for each sensor:")
    print(sensor_frequency)

    # Return two dataframes one with values above a certain threshold and another with summary statistics
    # Get a DataFrame where the difference between logging times is more than thres_mins minutes
    filtered_df = df[df['TimeDiff'].dt.total_seconds() > (thres_mins*60)]

    # Group the DataFrame by sensor and calculate the desired statistics
    summary_df = df.groupby('Name').agg({'Value': ['count', 'sum', 'mean']})

    # Rename the columns
    summary_df.columns = ['Count', 'Sum', 'Mean']

    return summary_df, filtered_df, df

def select_data_range_values(start_date, end_date, df, col='Date', details=False):
    """This function slices off data for a given date rnage where the start and end date are specified"""
    # Slice off the dataframe matching this condition
    sliced_df = df[(df[col] >= start_date) & (df[col] <= end_date)]

    if details:
        # Check if there are some missing data
        start, end = date_range(sliced_df, col)

        if start != start_date:
            print(f'Mismatch start: Expected {start_date} vs Gotten (from slice) {start}')
        if end != end_date:
            print(f'Mismatch end: Expected {end_date} vs Gotten (from slice) {end}')

    return sliced_df


def smooth_data(df, avg='5'):
    """This function takes a dataframe object and computes 5 minutes average which is based on the frequency of the reference data 
    it returns this smoothed average"""
    
    # set the date column as the index
    df = df.set_index('Date')

    # calculate the average over 5 minutes
    df_avg = df['Value'].resample(f'{avg}T').mean()

    # Reset the index
    df_avg = df_avg.reset_index()

    # return the averaged df
    return df_avg


def precise_frequency(df, col='Date'):    
    # calculate time difference between consecutive datetimes
    time_diffs = df[col].diff()

    # Get the frequency of measuremeant as a mean
    sensor_frequency = time_diffs.mean()

    # count the number of occurrences of each time difference
    freq_counts = time_diffs.value_counts()
    print(f"Frequency counts \n{freq_counts}")

    return pd.to_timedelta(sensor_frequency).round('T'), freq_counts


def plot_date_range(combined_df,start_date,end_date,features_list,title,unit,show_diff=False):
    """
    This function helps to plot Sensors data for a specific data range and their difference.
    it can be used following this example:
    temperature_data_df.loc[start_date:end_date,['hdc_1', 'hdc_2','hdc_average',"temperature S001544 (degrees Celsius)"]].iplot(title= "Monthly HDC sensor temperature data", xTitle= "Time", yTitle= "degrees Celsius")
    """
    if(show_diff):
        return combined_df.loc[start_date:end_date,features_list].iplot( kind = "spread", 
                                                                        title="{} Sensors Data from {} to {} and their Difference".format(title, start_date,end_date),
                                                                        xTitle="Time",
                                                                        yTitle= unit) 

    return combined_df.loc[start_date:end_date,features_list].iplot( title="{} Sensors Data from {} to {}".format(title, start_date,end_date),
                                                                     xTitle="Time",
                                                                     yTitle= unit)

In [4]:
temperature_df = pd.read_excel("WeatherStation Data - 20-06-2023.xlsx", sheet_name='Temperature')
temperature_df = convert_todatetime(temperature_df)
temperature_df

,Date,Time,Name,Value
0,2021-10-04 10:43:17,13:43:17,sht_1,29.07000
1,2021-10-04 10:43:17,13:43:17,sht_2,28.94000
2,2021-10-04 10:43:17,13:43:17,sht_3,29.11000
3,2021-10-04 10:43:17,13:43:17,bme_1,29.30000
4,2021-10-04 10:43:17,13:43:17,bme_2,28.48000
...,...,...,...,...
723179,2023-06-20 20:54:16,23:54:16,htu_1,0.00000
723180,2023-06-20 20:54:16,23:54:16,htu_2,23.16344
723181,2023-06-20 20:54:16,23:54:16,htu_3,21.59757
723182,2023-06-20 20:54:16,23:54:16,hdc_1,21.97571


In [5]:
# Temperature data information

print("Temperature data information")
print("============================")

# Get date ranges
temp_start, temp_end = date_range(temperature_df, 'Date')
print(f'Start date: {temp_start} \nEnd date: {temp_end}')

# Get the frequency of measurement
freq = get_frequency(temperature_df)

Temperature data information
Start date: 2021-10-04 10:43:17 
End date: 2023-06-20 20:54:16
Frequency of logging for each sensor:
Name
bme_1   0 days 00:13:00
bme_2   0 days 00:14:00
bme_3   0 days 00:14:00
hdc_1   0 days 00:14:00
hdc_2   0 days 00:15:00
htu_1   0 days 00:14:00
htu_2   0 days 00:14:00
htu_3   0 days 00:14:00
sht_1   0 days 00:13:00
sht_2   0 days 00:13:00
sht_3   0 days 00:13:00
Name: TimeDiff, dtype: timedelta64[ns]


In [6]:
reference_data_df = pd.read_csv("processedData/TA00216.csv") 
reference_data_df = strip_time(reference_data_df, 'timestamp (UTC)')
reference_data_df = reference_data_df[["timestamp (UTC)", "relativehumidity S001544 (-)" ,"temperature S001544 (degrees Celsius)"]].dropna().reset_index(drop=True)
end_data = reference_data_df["timestamp (UTC)"].tail(1).tolist()[0]



second_ref = pd.read_csv("RH-AT Data Only/TA00216.csv")
second_ref = second_ref.rename(columns={'timestamp': 'timestamp (UTC)'})
second_ref = strip_time(second_ref, 'timestamp (UTC)')
second_ref = second_ref[["timestamp (UTC)", "relativehumidity S001544 (-)" ,"temperature S001544 (degrees Celsius)"]].dropna().reset_index(drop=True)
second_ref = second_ref.loc[second_ref["timestamp (UTC)"] > end_data].reset_index(drop=True)


reference_data_df = pd.concat([reference_data_df, second_ref], axis=0).reset_index(drop=True)
reference_data_df

,timestamp (UTC),relativehumidity S001544 (-),temperature S001544 (degrees Celsius)
0,2021-02-03 11:10:00,0.436,29.8
1,2021-02-03 11:15:00,0.436,29.8
2,2021-02-03 11:20:00,0.455,29.8
3,2021-02-03 11:25:00,0.461,29.6
4,2021-02-03 11:30:00,0.481,29.0
...,...,...,...
189981,2023-03-13 09:45:00,0.616,27.3
189982,2023-03-13 09:50:00,0.611,27.0
189983,2023-03-13 09:55:00,0.608,27.2
189984,2023-03-13 10:00:00,0.610,27.1


In [7]:
# humidity data information
print("Reference data information")
print("==========================")

ref_start, ref_end = date_range(reference_data_df , 'timestamp (UTC)')
print(f'Start date: {ref_start} \nEnd date: {ref_end}')

reference_data_df = strip_time(reference_data_df, 'timestamp (UTC)')

freq = precise_frequency(reference_data_df, 'timestamp (UTC)')
print(f'Frequency: {freq[0]} minutes')

Reference data information
Start date: 2021-02-03 11:10:00 
End date: 2023-03-13 10:05:00
Frequency counts 
0 days 00:05:00     189914
0 days 00:10:00         32
0 days 23:10:00          2
0 days 19:00:00          2
0 days 07:55:00          2
0 days 00:15:00          2
0 days 19:15:00          1
0 days 14:45:00          1
0 days 05:10:00          1
5 days 20:15:00          1
0 days 15:05:00          1
3 days 20:10:00          1
0 days 03:55:00          1
0 days 01:20:00          1
0 days 19:55:00          1
0 days 13:00:00          1
0 days 17:05:00          1
0 days 16:25:00          1
0 days 14:25:00          1
8 days 22:00:00          1
0 days 21:15:00          1
0 days 20:20:00          1
0 days 00:30:00          1
0 days 20:40:00          1
0 days 19:40:00          1
0 days 22:55:00          1
0 days 20:25:00          1
0 days 05:40:00          1
2 days 00:15:00          1
1 days 06:50:00          1
0 days 21:05:00          1
1 days 02:00:00          1
1 days 00:05:00          1
0

In [8]:
sliced_temp = select_data_range_values(ref_start, ref_end, temperature_df)


temp_hum_ref = select_data_range_values(temp_start, temp_end, reference_data_df, 'timestamp (UTC)')

In [9]:
sliced_temp

,Date,Time,Name,Value,TimeDiff
0,2021-10-04 10:43:17,13:43:17,sht_1,29.07000,NaT
1,2021-10-04 10:43:17,13:43:17,sht_2,28.94000,NaT
2,2021-10-04 10:43:17,13:43:17,sht_3,29.11000,NaT
3,2021-10-04 10:43:17,13:43:17,bme_1,29.30000,NaT
4,2021-10-04 10:43:17,13:43:17,bme_2,28.48000,NaT
...,...,...,...,...,...
602135,2023-03-12 08:55:33,11:55:33,htu_1,0.00000,0 days 00:05:11
602136,2023-03-12 08:55:33,11:55:33,htu_2,18.74470,0 days 00:05:11
602137,2023-03-12 08:55:33,11:55:33,htu_3,18.03684,0 days 00:05:11
602138,2023-03-12 08:55:33,11:55:33,hdc_1,18.43079,0 days 00:05:11


In [10]:
temp_hum_ref 

,timestamp (UTC),relativehumidity S001544 (-),temperature S001544 (degrees Celsius)
63358,2021-10-04 10:45:00,0.576,28.8
63359,2021-10-04 10:50:00,0.591,28.8
63360,2021-10-04 10:55:00,0.580,28.5
63361,2021-10-04 11:00:00,0.569,28.4
63362,2021-10-04 11:05:00,0.591,28.5
...,...,...,...
189981,2023-03-13 09:45:00,0.616,27.3
189982,2023-03-13 09:50:00,0.611,27.0
189983,2023-03-13 09:55:00,0.608,27.2
189984,2023-03-13 10:00:00,0.610,27.1


In [11]:
temp_sensor_dfs = {sensor: group for sensor, group in sliced_temp.groupby('Name')}
assert len(temp_sensor_dfs) == len(sliced_temp['Name'].unique())

sample_row = sliced_temp.sample(1)
key = sample_row.values[0, 3]

if key in temp_sensor_dfs:
    selected_group = temp_sensor_dfs[key]
    print(selected_group)
else:
    print("Key does not exist in temp_sensor_dfs dictionary.")

Key does not exist in temp_sensor_dfs dictionary.


In [12]:
# display all sensors available
print(f"All sensors available: {sliced_temp['Name'].unique()}")

All sensors available: ['sht_1' 'sht_2' 'sht_3' 'bme_1' 'bme_2' 'bme_3' 'htu_1' 'htu_2' 'htu_3'
 'hdc_1' 'hdc_2']


In [13]:
for sensor in temp_sensor_dfs.keys():
    temp_sensor_dfs[sensor] = smooth_data(temp_sensor_dfs[sensor])
    precise_frequency(temp_sensor_dfs[sensor])

Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64
Frequency counts 
0 days 00:05:00    150891
Name: Date, dtype: int64


In [14]:
temp_sensor_dfs

{'bme_1':                       Date   Value
 0      2021-10-04 10:40:00  29.300
 1      2021-10-04 10:45:00  29.560
 2      2021-10-04 10:50:00  29.650
 3      2021-10-04 10:55:00  29.745
 4      2021-10-04 11:00:00  29.570
 ...                    ...     ...
 150887 2023-03-12 08:35:00     NaN
 150888 2023-03-12 08:40:00  18.430
 150889 2023-03-12 08:45:00  18.310
 150890 2023-03-12 08:50:00  18.380
 150891 2023-03-12 08:55:00  18.340
 
 [150892 rows x 2 columns],
 'bme_2':                       Date   Value
 0      2021-10-04 10:40:00  28.480
 1      2021-10-04 10:45:00  28.680
 2      2021-10-04 10:50:00  28.860
 3      2021-10-04 10:55:00  28.970
 4      2021-10-04 11:00:00  28.925
 ...                    ...     ...
 150887 2023-03-12 08:35:00     NaN
 150888 2023-03-12 08:40:00  20.310
 150889 2023-03-12 08:45:00  20.140
 150890 2023-03-12 08:50:00  20.200
 150891 2023-03-12 08:55:00  20.040
 
 [150892 rows x 2 columns],
 'bme_3':                       Date   Value
 0      2021-

In [15]:
temp_hum_ref

In [16]:
temperature_data_df = temp_hum_ref[["timestamp (UTC)","temperature S001544 (degrees Celsius)"]].reset_index(drop=True)
temperature_data_df= temperature_data_df.rename(columns={'timestamp (UTC)': 'Date'})

for sensor in temp_sensor_dfs.keys():
    new_df = temp_sensor_dfs[sensor][["Date", "Value"]]
    
    temperature_data_df = temperature_data_df.merge(new_df, on="Date", how='outer')
    temperature_data_df = temperature_data_df.rename(columns={"Value": sensor})
    
temperature_data_df = temperature_data_df.sort_values(by="Date").reset_index(drop=True)
temperature_data_df = temperature_data_df.dropna()

In [17]:
temperature_data_df["Reference Data"] = temperature_data_df["temperature S001544 (degrees Celsius)"]

In [18]:
temperature_data_df = temperature_data_df.drop(columns='temperature S001544 (degrees Celsius)')

In [19]:
temperature_data_df

In [20]:
# temperature_data_df.to_csv('TemperatureData.csv', index=False)

In [21]:
len(temperature_data_df)